In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from functions import *

In [ ]:
from qibo.symbols import X, Z

In [ ]:
nqubits = 6

In [ ]:
folder = 'qubo_small'

typeM = 'qiskitM'
files_qiskit = get_filenames(folder, typeM, nqubits)

typeM = 'ourM'
files_our = get_filenames(folder, typeM, nqubits)


solutions = get_filenames(folder, 'solution', nqubits)

In [ ]:
# Debug

In [ ]:
x = symbols(" ".join((f"x{i}" for i in range(0, nqubits))))

dict_x = {str(xx):xx for xx in x}

sym_obj = get_sym_obj(files_our[0], dict_x)
sym_ham = get_sym_ham(sym_obj, x)
sol, sol_bin = get_solution(solutions[0], nqubits)

In [ ]:
sol

In [ ]:
s = lambda t: t

In [ ]:
ham0 = get_h0(nqubits)

In [ ]:
h0 = sum((0.5 * (1 - X(i))) for i in range(nqubits))

In [ ]:
subs = substitutions(x)
ham = simplify(sym_obj.subs(subs))
max_eig, min_eig = get_maxmin_eigenvalues(ham)
ham = simplify((ham-min_eig)/(max_eig-min_eig))

In [ ]:
# Check the gap during the evolution

x = np.linspace(0, 1, 500)
energies = []
for t in x:
    h = (1-t)*h0/nqubits+t*ham
    h = hamiltonians.SymbolicHamiltonian(h)
    eigs = h.eigenvalues()
    energies.append(eigs)
    

In [ ]:
energies = np.array(energies).T

In [ ]:
# Plot the energies for different value of s
for en in energies[:3]:
    plt.plot(x[-20:], en[-20:])

In [ ]:
# Plot the gap for different values of s
plt.plot(x[-20:], np.array(energies[1][-20:])-np.array(energies[0][-20:]))

In [ ]:
# Try an instance of adiabatic evolution
evolve = AdiabaticEvolution(ham0, sym_ham, s, dt=1, solver='exp')
final_state = evolve(final_time=1000000, initial_state=np.copy(ground))

In [ ]:
# Plot some of the results to see how well it works

In [ ]:
final_state

In [ ]:
sym_ham.expectation(final_state)

In [ ]:
np.abs(final_state).argmax()

In [ ]:
(np.abs(final_state)**2).max()

In [ ]:
np.sum(np.abs(final_state)**2)

In [ ]:
sol

In [ ]:
# Run the comparison algorithm for a set of the files
probs_our, probs_qiskit = compare_adiabatic(files_our[:100], files_qiskit[:100], solutions[:100], 1000000, nqubits, dt=10)
save_data(probs_our, 'adiabatic_our_T_100_dt_.5')
save_data(probs_qiskit, 'adiabatic_qiskit_T_100_dt_.5')

In [ ]:
ratio = probs_our/probs_qiskit

In [ ]:
print(np.sum(ratio >= 1))

In [ ]:
plt.plot(np.sort(ratio))

In [ ]:
nparams = 10

In [ ]:
# Run a similar comparison but with qaoa only
probs_our, probs_qiskit = compare_qaoa(files_our, files_qiskit, solutions, nqubits, nparams)
save_data(probs_our, 'qaoa_our_LBFGS_2qubits_parameters')
save_data(probs_qiskit, 'qaoa_qiskit_LBSGS_2qubits_parameters')

In [ ]:
ratio = probs_our/probs_qiskit

In [ ]:
plt.plot(np.sort(ratio))